# Multi-Scenario MORDM

Multi-scenario MORMD is an extension of normal MORDM to better include robustness considerations within the search phase. It starts from the scenario discovery results resulting from MORDM. Next, from the experiments within this box, a set of scenarios is selected. 

There are many ways of selecting the additional scenarios. The original paper which introduced multi-scenario MORMD [Watson and Kaspzryk (2017)](https://doi.org/10.1016/j.envsoft.2016.12.001) did it in a more or less adhoc manner. [Eker and Kwakkel (2018)](https://doi.org/10.1016/j.envsoft.2018.03.029) introduced a more formal selection approach, the code of which can be found on [GitHub](https://github.com/sibeleker/MORDM---Multi-scenario-search). 

For this assignment, make an informed selection of 4 scenarios, using an approach of your choice. Motivate carefully your selection procedure. 


In [1]:
import ema_workbench
from ema_workbench import save_results, load_results

from ScenarioSelection_v2 import normalize_out_dic, evaluate_diversity_single, find_maxdiverse_scenarios

results = load_results('./selected_results.tar.gz')
experiments, outcomes = results
results

(            b     delta      mean         q     stdev        c1        c2  \
 0    0.364981  0.945650  0.042731  3.842727  0.002098  0.356101  1.307652   
 1    0.189104  0.952855  0.035393  4.283846  0.003653  0.356101  1.307652   
 2    0.178390  0.957969  0.010711  3.472466  0.003666  0.356101  1.307652   
 3    0.403766  0.961101  0.038864  3.498893  0.003139  0.356101  1.307652   
 4    0.201551  0.946347  0.029143  3.022100  0.001885  0.356101  1.307652   
 ..        ...       ...       ...       ...       ...       ...       ...   
 689  0.248843  0.952572  0.012788  3.381476  0.003681  0.274703  0.978422   
 690  0.278984  0.956398  0.018040  3.982658  0.003427  0.274703  0.978422   
 691  0.219555  0.961186  0.039711  4.219227  0.004017  0.274703  0.978422   
 692  0.296006  0.934129  0.023829  4.260191  0.004595  0.274703  0.978422   
 693  0.214287  0.941883  0.045099  4.403661  0.003054  0.274703  0.978422   
 
            r1        r2        w1 scenario policy        mode

## Search for each scenario

For each of the four selected scenarios, use many-objective optimization to find a pareto approximate set using the same approach as for assignment 8. Remember to check for convergence (and time permitting, seed analysis), and be careful in what epsilon values to use (not to coarse, not too small). 

Store the resulting set of pareto solutions in a smart way for subsequent analysis.


In [2]:
oois = sorted(outcomes.keys())

In [43]:
# max pollution
import operator

stats = outcomes['max_P'].max()
outcome_P0 = np.where(outcomes['max_P'] == stats)
experiments_P0 = experiments.loc[outcome_P0[0],:]


#due to time constraints we were unable to complete other scenarios, we chose max pollution due to time constrait as well
# max pollution
# max reliability
# min reliability

In [71]:
experiments_P0

,b,delta,mean,q,stdev,c1,c2,r1,r2,w1,scenario,policy,model
160,0.13338,0.943518,0.040424,3.319402,0.001061,0.356101,1.307652,0.62284,1.986887,0.95986,478,1,lakeproblem


In [84]:
from dps_lake_model import lake_model, get_antropogenic_release

from ema_workbench import (Model, RealParameter, ScalarOutcome,
                           MultiprocessingEvaluator, ema_logging,
                           Constant, Scenario)
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
from ema_workbench import Constraint


ema_logging.log_to_stderr(ema_logging.INFO)

#instantiate the model
model = Model('lakeproblem', function=lake_model)
model.time_horizon = 100 # used to specify the number of timesteps

#specify uncertainties
model.uncertainties = [RealParameter('mean', 0.01, 0.05),
                            RealParameter('stdev', 0.001, 0.005),
                            RealParameter('b', 0.1, 0.45),
                            RealParameter('q', 2.0, 4.5),
                            RealParameter('delta', 0.93, 0.99)]

# set levers, one for each time step
model.levers = [RealParameter('c1', -2, 2),
                            RealParameter('c2', -2, 2),
                            RealParameter('r1', 0, 2),
                            RealParameter('r2', 0, 2),
                            RealParameter('w1', 0, 1)]
#specify outcomes 
model.outcomes = [ScalarOutcome('max_P', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,5)),
                       ScalarOutcome('utility', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,2)),
                       ScalarOutcome('inertia', kind=ScalarOutcome.MAXIMIZE,
                                    expected_range=(0,1)),
                       ScalarOutcome('reliability', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,1))]

model.constantcs = [Constant('alpha', 0.41),
                         Constant('reps', 150)]

convergence_metrics = [HyperVolume.from_outcomes(model.outcomes),
                       EpsilonProgress()]

constraints = [Constraint("max pollution", outcome_names="max_P",
                          function=lambda x:max(0, x-5))]

In [85]:
ref = experiments_P0['scenario'].astype(int)

reference = Scenario('reference', scenario=ref)

# reference = Scenario('reference', b=experiments_P0['b'], q=experiments_P0['q'], mean=experiments_P0['mean'] stdev=experiments_P0['st'])

In [92]:
with MultiprocessingEvaluator(model) as evaluator:
    results0, convergence = evaluator.optimize(nfe=100, reference = reference,
                                    epsilons=[0.1, 0.1, 0.1, 0.1],
                                    convergence=convergence_metrics,
                                    constraints=constraints)

[MainProcess/INFO] generation 0: 0/100 nfe
[MainProcess/INFO] optimization completed, found 8 solutions


In [93]:
results0

,c1,c2,r1,r2,w1,max_P,utility,inertia,reliability
0,0.880450,0.573012,1.520647,1.932274,0.502613,2.283618,0.775724,0.9694,0.4500
1,0.370061,0.566246,1.657987,1.788683,0.714301,0.091302,0.214257,0.9900,1.0000
2,0.724811,0.025060,1.538812,0.474227,0.200626,2.283628,1.008461,0.9771,0.3600
3,0.197182,-0.156253,1.192239,0.944618,0.031843,2.283914,1.255716,0.9815,0.2296
4,-0.005540,0.815401,1.814442,1.391004,0.772529,0.170809,0.460728,0.9900,1.0000
5,1.801131,0.981740,0.465998,0.081860,0.694811,2.283685,1.734761,0.9900,0.0700
6,1.077338,0.306297,1.716885,0.295024,0.483124,2.283929,1.582701,0.9712,0.1200
7,-0.306577,0.472583,1.187430,0.863503,0.108302,0.252700,0.554703,0.9900,1.0000


We were unable to complete this assignment due to time constraint and preperation for the debate - for this reason we did a manual scenario selection for only 1 (max_P) rather than a formal search over 4

## Re-evaluate under deep uncertainty

Combine the pareto set of solutions found for each scenario. Next, turn each solution into a policy object. If you have a very large number of policies, you can choose to down sample your policies in some reasoned way (*e.g.*, picking min and max on each objective, slicing across the pareto front with a particular step size). As a rule of thumb, try to limit the set of policies to at most 50. 

Re-evaluate the combined set of solutions over 1000 scenarios sampled using LHS.


Calculate both the maximum regret, and the domain criterion using the values provided in [Bartholomew and Kwakkel (2020)](https://doi.org/10.1016/j.envsoft.2020.104699). Ignore the max_P objective.

visualize the results in parallel coordinate plot. 

Are there any promising compromise solutions which balance performance in both the reference scenarios as well as in terms of their robustness?
